# Training: GDrive x BigQuery x GSheets
This notebook provides guidance on how you can connect your [Google Colab](https://colab.research.google.com/notebooks/welcome.ipynb#) instance to the following:

- Google Drive (GDrive) 
- BigQuery (BQ)
- Google Sheets (GSheets)

The interaction of these three is useful if you want to extract data from BigQuery and put it into a Google Sheet.

***

## Contents

The outline of this notebook is:

1. Mount GDrive
1. Connect to BigQuery
    + Import data from there
1. Prepare your data
1. Open and save into GSheets